# Bulk Functions

This notebook demonstrates how to access and use the functinalities of **Bulk functions** which are part of our **YieldBook** module within LSEG Financial Analytics SDK.

## Imports

Import the following necessary modules:

- bulk_*** - libraries to access various bulk functionalities
- get_results - get singular request results
- get_csv_bulk_result - get a multiple request results at once
- json - used for printout formating
- time - used for response delay for call execution

In [1]:
from lseg_analytics.pricing.yield_book_rest import (
    bulk_compact_request,
    bulk_composite_request,
    BulkJsonInputItem,
    get_result,
    get_csv_bulk_result,
    get_job_status
)

import json as js

import pandas as pd
from io import StringIO

import time

## Bulk Compact Request

A bulk request can be done in several ways, the first of which is using the Compact format where the user has to provide the following information:
- path (str, required) - URL to which each individual request should be posted.i.e “/bond/py” for PY calculation.
    - Currently supported paths:
        - "/bond/indic" - Used for retrieval of Indicative data
        - "/bond/py" - Used for retrieval of PriceYield data
        - "/bond/cash-flow" - Used for retrieval of Cash Flow data
- name_expr (str, optional) - Name of each request. This can be a valid JSON path expression, i.e “concat($.CUSIP,”_PY”)” will give each request the name CUSIP_PY. Name should be unique within a single job.
- body (str, required) – POST body associated with the calculation. This is specific to each request type. Refer to individual calculation documentation for more details.
- requests (List[Dict[str, Any]], required) – List of key value pairs. This values provided will be used to update corresponding variables in the body of the request.
- params (Dict[str, Any], optional) - List of additional parameters for calculations that support it
- create_job (bool, exclusively required) – Boolean with true and false values. Exclusively required with 'job' argument.
- job (str, exclusively required) – Job reference. This can be of the form J-number or job name. Exclusively required with 'create_job'
- name (str, optional) – User provided name.
- pri (int, optional) – Priority for this request in the queue.
- tags (List[str], optional) - List of tags used for subsequent addressing.

In [2]:
compact_job_name = "J-123456"

# bulk_compact_request
compact_response = bulk_compact_request(
                    path="/bond/py",
                    name_expr='concat($.CUSIP,"_PY")',
                    body=
                        {
                            "globalSettings": 
                                { 
                                    "pricingDate": "2023-03-31" 
                                }, 
                            "input": 
                                [ 
                                    { 
                                        "identifier.$": "$.id", 
                                        "idType": "cusip", 
                                        "level.$": "$.level", 
                                        "curve": 
                                            {
                                                "curveType": "SWAP"
                                            }, 
                                        "volatility": 
                                            {
                                                "type": "Default"
                                            }
                                    }
                                ]
                        },
                    requests=
                        [
                            {
                                "id": "31418DY55", 
                                "level": "96.578"
                            }, 
                            {   
                                "id": "31418D4J8", 
                                "level": "96.131"
                            }
                        ],
                    params={},
                    create_job=True,
                    job=compact_job_name,
                    name="BulkCompactRun",
                    pri=0
                )

print("Created following requests:")

# Iterate through all request ID's
for result in compact_response.results:
    print(result.id)

Created following requests:
R-150615
R-150616


## Compact results retrieval and display

When a bulk operation is performed, the user is provided with a Request ID (visible in the previous output).

These IDs can be used to obtain the results of the execution in one of two ways:
- Singular - using get_results function with a single ID
- Bulk - using get_csv_bulk_results function with a list of IDs

### Singular Result

In [3]:
# Due to async nature, code Will perform the job status check 10 times, as result is not always ready instantly, with 10 second lapse between attempts
print("Calculating please wait...\n")

attempt = 1
max_attempts = 10
all_compact_results = []
 
while attempt < max_attempts:
        
        response = get_job_status(job_ref=compact_job_name)
        ok = response.get("okCount", None)
        pending = response.get("pendingCount", None)
        running = response.get("runningCount", None)
        if running == 0 and pending == 0 and ok != 0:
            print("Job is complete!\n")
            for result in compact_response.results:
                # Request results
                singular_compact_result = get_result(request_id_parameter=result.id)
                all_compact_results.append(singular_compact_result)
            break
        else:
              # If job is not yet done, repeat the loop
              print(response)
              time.sleep(10)
              attempt += 1

# Print results
print(js.dumps(all_compact_results, indent=4))

Calculating please wait...

{'id': 'J-16437', 'name': 'J-123456', 'jobStatus': 'CLOSED', 'pendingCount': -2, 'runningCount': 2}
Job is complete!

[
    {
        "meta": {
            "status": "DONE",
            "requestId": "R-150615",
            "timeStamp": "2025-12-03T07:43:28Z",
            "responseType": "PY_CALC",
            "resultsStatus": "ALL"
        },
        "results": [
            {
                "py": {
                    "oas": -127.9353,
                    "wal": 3.066098,
                    "dv01": 0.027898833,
                    "cusip": "31418DY55",
                    "price": 96.578,
                    "yield": 2.663111,
                    "ticker": "FNCN",
                    "cmmType": 100,
                    "eSpread": -125.1711,
                    "pyLevel": "96.578",
                    "zSpread": -120.395966,
                    "duration": 2.886743,
                    "ziSpread": -107.852348,
                    "znSpread": -120.395966,
 

### Bulk Results

In [4]:
# Due to async nature, code Will perform the job status check 10 times, as result is not always ready instantly, with 10 second lapse between attempts
print("Calculating please wait...\n")

attempt = 1
max_attempts = 10
 
while attempt < max_attempts:
        
        response = get_job_status(job_ref=compact_job_name)
        ok = response.get("okCount", None)
        pending = response.get("pendingCount", None)
        running = response.get("runningCount", None)
        if running == 0 and pending == 0 and ok != 0:
            print("Job is complete! \n")
            bulk_compact_result_csv = get_csv_bulk_result(
                                                ids=[result.id for result in compact_response.results], 
                                                fields=["CUSIP:py.cusip", "Yield:py.yield", "OAS:py.oas"]
                                            )
            break
        else:
              # If job is not yet done, repeat the loop
              print(response)
              time.sleep(10)
              attempt += 1

# Print Bulk singular result
df = pd.read_csv(StringIO(bulk_compact_result_csv))
print(df)

Calculating please wait...

Job is complete! 

       CUSIP     Yield       OAS
0  31418DY55  2.663111 -127.9353
1  31418D4J8  2.781447 -115.1572


## Bulk Composite Request

The second way to execute a bulk request is in using the Composite format, where a user has to provide the following information:

- requests (List[BulkJsonInputItem], required) – List of BulkJsonInputItem items. Please consult documentation for 
- create_job (bool, exclusively required) – Boolean with true and false values. Exclusively required with 'job' argument.
- job (str, exclusively required) – Job reference. This can be of the form J-number or job name. Exclusively required with 'create_job'
- name (str, optional) – User provided name.
- pri (int, optional) – Priority for this request in the queue

With the Composite function, unlike the Compact function, the user can set the input valus for each individual instrument.

In [5]:
composite_job_name = "J-654321"

# bulk_composite_request
composite_response = bulk_composite_request(
                        requests=[
                            BulkJsonInputItem(
                                path="/bond/py",
                                body=
                                    {
                                        "globalSettings": 
                                            { 
                                                "pricingDate": "2023-03-31"
                                            },
                                        "input": 
                                            [ 
                                                {
                                                    "identifier": "31418DY55",
                                                    "idType": "cusip", 
                                                    "level": "109.706", 
                                                    "curve": 
                                                        {
                                                            "curveType": "SWAP"
                                                        }, 
                                                    "volatility": 
                                                        {
                                                            "type": "Default" 
                                                        }
                                                },                                                
                                                {
                                                    "identifier": "31418D4J8",
                                                    "idType": "cusip", 
                                                    "level": "96.131", 
                                                    "curve": 
                                                        {
                                                            "curveType": "SWAP"
                                                        }, 
                                                    "volatility": 
                                                        {
                                                            "type": "Default" 
                                                        }
                                                }
                                            ]
                                    },
                            )
                        ],
                        create_job=True,
                        job=composite_job_name
                    )

print("Created following requests:")

# Iterate through all request ID's
for result in composite_response.results:
    print(result.id)

Created following requests:
R-150618


## Composite results retrieval and display

In the Composite bulk function context, the user is presented with a single request ID, which can be used with both result retrieval functions, either for a single result or for bulk results with multiple request IDs.

### Bulk result retrieval - get_result()

In [6]:
# Due to async nature, code Will perform the job status check 10 times, as result is not always ready instantly, with 10 second lapse between attempts
print("Calculating please wait...\n")

attempt = 1
max_attempts = 10
all_composite_results = []
 
while attempt < max_attempts:
        
        response = get_job_status(job_ref=composite_job_name)
        ok = response.get("okCount", None)
        pending = response.get("pendingCount", None)
        running = response.get("runningCount", None)
        if running == 0 and pending == 0 and ok != 0:
            print("Job is complete!\n")
            for result in composite_response.results:
                # Request results
                singular_composite_result = get_result(request_id_parameter=result.id)
                all_composite_results.append(singular_composite_result)
            break
        else:
              # If job is not yet done, repeat the loop
              print(response)
              time.sleep(10)
              attempt += 1

# Print results
print(js.dumps(all_composite_results, indent=4))

Calculating please wait...

{'id': 'J-16438', 'name': 'J-654321', 'jobStatus': 'CLOSED', 'requestCount': 1, 'pendingCount': 0, 'runningCount': 1, 'okCount': 0, 'errorCount': 0, 'abortedCount': 0, 'skippedCount': 0}
Job is complete!

[
    {
        "meta": {
            "status": "DONE",
            "requestId": "R-150618",
            "timeStamp": "2025-12-03T07:43:41Z",
            "responseType": "PY_CALC",
            "resultsStatus": "ALL"
        },
        "results": [
            {
                "py": {
                    "oas": -548.3548,
                    "wal": 3.066098,
                    "dv01": 0.034245618,
                    "cusip": "31418DY55",
                    "price": 109.706,
                    "yield": -1.580365,
                    "ticker": "FNCN",
                    "cmmType": 100,
                    "eSpread": -540.201,
                    "pyLevel": "109.706",
                    "zSpread": -541.84021,
                    "duration": 3.119685,
   

### Bulk result retrieval - get_csv_bulk_result()

In [7]:
# Due to async nature, code Will perform the job status check 10 times, as result is not always ready instantly, with 10 second lapse between attempts
print("Calculating please wait...\n")

attempt = 1
max_attempts = 10
all_composite_results = []
 
while attempt < max_attempts:
        
        response = get_job_status(job_ref=composite_job_name)
        ok = response.get("okCount", None)
        pending = response.get("pendingCount", None)
        running = response.get("runningCount", None)
        if running == 0 and pending == 0 and ok != 0:
            print("Job is complete!\n")
            bulk_composite_result_csv = get_csv_bulk_result(
                                                    ids=[result.id for result in composite_response.results], 
                                                    fields=["CUSIP:py.cusip", "Yield:py.yield", "OAS:py.oas"]
                                                )
            break
        else:
              # If job is not yet done, repeat the loop
              print(response)
              time.sleep(10)
              attempt += 1

# Print Bulk result
df = pd.read_csv(StringIO(bulk_composite_result_csv))
print(df)

Calculating please wait...

Job is complete!

       CUSIP     Yield       OAS
0  31418DY55 -1.580365 -548.3548
1  31418D4J8  2.781447 -115.1572
